In [49]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpInteger, LpStatus, value
import re

# Toggles for the dietary restrictions
only_vegan = False
only_vegetarian = False
consider_vitamins = True
consider_minerals = True
gender = "Women"  # or "Men"
age_group = "19-30"  # one of "19-30", "31-50", "51-70", "71+"

min_col = f"{gender} {age_group}"
limit_col = f"Limit {age_group}"

# Load food, vitamin, and mineral data CSV
food_data = pd.read_csv('food_data.csv')
vitamin_constraints = pd.read_csv("vitamin_constraints.csv")
mineral_constraints = pd.read_csv("mineral_constraints.csv") 

# Filter the data based on vegan/vegetarian toggles
if only_vegan:
    food_data = food_data[food_data['vegan'] == True]

if only_vegetarian:
    food_data = food_data[food_data['vegetarian'] == True]


# Required amounts per unit P of complete protein
required = {
    "Histidine": 14,
    "Isoleucine": 19,
    "Leucine": 42,
    "Lysine": 38,
    "Methionine": 19,
    "Phenylalanine": 33,
    "Threonine": 20,
    "Tryptophan": 5,
    "Valine": 24
}

calorie_limit = 2000

prob = LpProblem("Maximize_Complete_Protein", LpMaximize)

# Decision variables: x[i] = number of 100 mg portions of food i
food_names = food_data['Name'].tolist()

# Turns into integer problem
#x = LpVariable.dicts("Portions", food_names, lowBound=0, cat=LpInteger)
x = LpVariable.dicts("Portions", food_names, lowBound=0)  # Defaults to continuous

# P = amount of complete proteins formed
P = LpVariable("P", lowBound=0)

# Objective is to maximize P
prob += P, "Maximize the number of complete proteins"

# Amino acid constraints
for aa in required.keys():
    prob += lpSum([x[row['Name']] * row[aa] for _, row in food_data.iterrows()]) >= required[aa] * P, f"AminoAcid_{aa}"

# Calorie constraint
prob += lpSum([x[row['Name']] * row["Calories"] for _, row in food_data.iterrows()]) <= calorie_limit, "CalorieConstraint"

#Vitamin Constraint
if consider_vitamins:
    for _, row in vitamin_constraints.iterrows():
        vitamin_name = row['Vitamin']  #"A & Beta Carotene"
        var_name = row['Variable']     #"V1"
        
        vitamin_columns = []
        if vitamin_name == "A & Beta Carotene":
            vitamin_columns = ["Vitamin A", "Beta-Carotene"]
        else:
            if vitamin_name in food_data.columns:
                vitamin_columns = [vitamin_name]
    
        # Get min and max for this vitamin
        vit_min = row[min_col]    #recommended intake
        vit_max = row[limit_col]  #upper limit
    
        for vcol in vitamin_columns:
            if vcol in food_data.columns:
                # Create an expression for the vitamin intake: sum of x[i]*vitamin_amount_in_food_data
                vitamin_expr = lpSum([x[f] * food_data.loc[food_data['Name'] == f, vcol].values[0] for f in food_names])
                
                # Min constraint
                prob += vitamin_expr >= vit_min, f"VitaminMin_{vcol}"
                
                if vit_max < 9999999:
                    prob += vitamin_expr <= vit_max, f"VitaminMax_{vcol}"

# Mineral constraints
if consider_minerals:
    for _, row in mineral_constraints.iterrows():
        mineral_name = row['Mineral']
        min_req = row[min_col]    
        max_req = row[limit_col]  

        if mineral_name in food_data.columns:
            mineral_expr = lpSum([x[f] * food_data.loc[food_data['Name'] == f, mineral_name].values[0] for f in food_names])
            
            # Minimum constraint
            prob += mineral_expr >= min_req, f"MineralMin_{mineral_name}"

            # Maximum constraint
            if max_req < 9999999:
                prob += mineral_expr <= max_req, f"MineralMax_{mineral_name}"

#Solve problem
prob.solve()

# Results
print("Status:", LpStatus[prob.status])
print("Optimal value of P (complete proteins):", value(P))
print("Selected foods (number of 100 mg portions):")
for v in prob.variables():
    if v.name.startswith("Portions") and value(v) > 0:
        print(v.name, "=", value(v))

Status: Optimal
Optimal value of P (complete proteins): 392.31778
Selected foods (number of 100 mg portions):
Portions_Basil = 0.18106109
Portions_Bass = 0.019805914
Portions_BayLeaf = 0.048801666
Portions_Carrrots = 0.062692042
Portions_CottageCheese = 0.94721688
Portions_Mussels = 0.10398596
Portions_Pumpkin = 0.60202671
Portions_SwissCheese = 0.65141631
Portions_Tuna = 3.890256
Portions_Veal = 6.5013631
